In [1]:
import warnings
warnings.filterwarnings("ignore")

import logging
import pandas as pd
import os
import gzip

from datetime import datetime
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

# importo los .py
import sys
sys.path.append("auxiliares")
import metricas

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
MES_TRAIN_TOPE = 201912
PERIODO_ESTACIONAL = 201902
PERIODO_MENOS_2 = 201912

In [4]:
arch_prod_ids_prediccion = "datasets/productos_a_predecir_201912.csv"
arch_sellout_productos = "datasets/emp3_sellout_producto_base.csv"

carpeta_base_exp = "exp/basica_naif/"

In [5]:
df_sellout = pd.read_csv(arch_sellout_productos)

In [6]:
df_sellout.head()

,product_id,periodo,product_id_tn,cero_ventas,cust_request_qty,cust_request_tn,mes,producto_estrella,plan_precios_cuidados,meses_historia_prod,cat1,cat2,cat3,sku_size,temp_media,temp_min_media,temp_max_media,IPC,promedio_mens_dolar_venta,catastrofe
0,20001,201701,934.77222,0,479.0,937.72717,1,1.0,0.0,0,HC,ROPA LAVADO,Liquido,3000.0,25.65,20.9,30.4,1.6,15.91,False
1,20001,201702,798.01620,0,432.0,833.72187,2,1.0,0.0,1,HC,ROPA LAVADO,Liquido,3000.0,25.35,20.6,30.1,2.1,15.59,False
2,20001,201703,1303.35771,0,509.0,1330.74697,3,1.0,0.0,2,HC,ROPA LAVADO,Liquido,3000.0,22.45,18.0,26.9,2.4,15.52,False
3,20001,201704,1069.96130,0,279.0,1132.94430,4,1.0,0.0,3,HC,ROPA LAVADO,Liquido,3000.0,18.90,14.3,23.5,2.7,15.36,False
4,20001,201705,1502.20132,0,701.0,1550.68936,5,1.0,0.0,4,HC,ROPA LAVADO,Liquido,3000.0,15.90,12.3,19.5,1.4,15.72,False


In [7]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34707 entries, 0 to 34706
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 34707 non-null  int64  
 1   periodo                    34707 non-null  int64  
 2   product_id_tn              34707 non-null  float64
 3   cero_ventas                34707 non-null  int64  
 4   cust_request_qty           34707 non-null  float64
 5   cust_request_tn            34707 non-null  float64
 6   mes                        34707 non-null  int64  
 7   producto_estrella          34707 non-null  float64
 8   plan_precios_cuidados      34707 non-null  float64
 9   meses_historia_prod        34707 non-null  int64  
 10  cat1                       34707 non-null  object 
 11  cat2                       34707 non-null  object 
 12  cat3                       34707 non-null  object 
 13  sku_size                   34707 non-null  flo

In [8]:
# Selecciono las columnas iniciales
df_sellout = df_sellout.rename(columns={"product_id_tn":"tn"})

df_sellout = df_sellout[["periodo","product_id","tn"]]

len(df_sellout.product_id.unique())

1233

## Train

In [9]:
df_train = df_sellout[df_sellout.periodo <= MES_TRAIN_TOPE]

In [10]:
print(len(df_train.product_id.unique()))
df_train

1233


,periodo,product_id,tn
0,201701,20001,934.77222
1,201702,20001,798.01620
2,201703,20001,1303.35771
3,201704,20001,1069.96130
4,201705,20001,1502.20132
...,...,...,...
34702,201907,21299,0.00000
34703,201909,21299,0.00000
34704,201910,21299,0.00000
34705,201911,21299,0.00000


In [11]:
df_prods_prediccion=pd.read_csv(arch_prod_ids_prediccion)
df_prods_prediccion.head()

,product_id
0,20001
1,20002
2,20003
3,20004
4,20005


In [12]:
len(df_prods_prediccion)

780

In [13]:
# Como puede haber productos que no estén en algunos períodos (que se usan en modelos naif), calculo promedio para completar
df_completar = df_train[["product_id","tn"]].groupby("product_id").mean().reset_index()

print(len(df_completar.product_id.unique()))
df_completar

1233


,product_id,tn
0,20001,1402.258580
1,20002,1014.956350
2,20003,896.244489
3,20004,677.029145
4,20005,647.272872
...,...,...
1228,21295,0.000200
1229,21296,0.000233
1230,21297,0.000165
1231,21298,0.000205


## Predicción Estacional

In [14]:
df_pred_estac = df_train[df_train.periodo==PERIODO_ESTACIONAL]
df_pred_estac = df_pred_estac[["product_id","tn"]]
df_pred_estac.head()

,product_id,tn
25,20001,1259.09363
60,20002,1043.01349
95,20003,758.32657
130,20004,441.70332
165,20005,409.89950


In [15]:
# Left join con prediccion promedio (para completar)
df_res_estac = pd.merge(df_completar,df_pred_estac,on="product_id",how="left",suffixes=["_prom","_estac"])

# Calculo toneladas predichas usando los valores anteriores
df_res_estac["tn_pred"]=np.where(df_res_estac.tn_estac.isnull(),df_res_estac.tn_prom,df_res_estac.tn_estac)

df_res_estac = df_res_estac[["product_id","tn_pred"]]
df_res_estac.head()

,product_id,tn_pred
0,20001,1259.09363
1,20002,1043.01349
2,20003,758.32657
3,20004,441.70332
4,20005,409.89950


In [16]:
df_res_estac.isna().sum()

product_id    0
tn_pred       0
dtype: int64

In [17]:
df_res_estac = df_res_estac[df_res_estac.product_id.isin(df_prods_prediccion.product_id)]

In [18]:
print(len(df_res_estac))
df_res_estac = df_res_estac.rename(columns={"tn_pred":"tn"})
df_res_estac.head()

780


,product_id,tn
0,20001,1259.09363
1,20002,1043.01349
2,20003,758.32657
3,20004,441.70332
4,20005,409.89950


In [19]:
df_res_estac.to_csv(carpeta_base_exp + "naif_estac_prediccion.csv",index=False)

## Predicción Menos 2

In [20]:
# Naif -2
df_pred_menos2 = df_train[df_train.periodo==PERIODO_MENOS_2]
df_pred_menos2 = df_pred_menos2[["product_id","tn"]]
df_pred_menos2.head()

,product_id,tn
34,20001,1504.68856
69,20002,1087.30855
104,20003,892.50129
139,20004,637.90002
174,20005,593.24443


In [21]:
# Left join con prediccion promedio (para completar)
df_res_menos2 = pd.merge(df_completar,df_pred_menos2,on="product_id",how="left",suffixes=["_prom","_menos2"])

# Calculo toneladas predichas usando los valores anteriores
df_res_menos2["tn_pred"]=np.where(df_res_menos2.tn_menos2.isnull(),df_res_menos2.tn_prom,df_res_menos2.tn_menos2)

df_res_menos2 = df_res_menos2[["product_id","tn_pred"]]
df_res_menos2.head()

,product_id,tn_pred
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443


In [22]:
df_res_menos2 = df_res_menos2[df_res_menos2.product_id.isin(df_prods_prediccion.product_id)]

In [23]:
print(len(df_res_menos2))
df_res_menos2 = df_res_menos2.rename(columns={"tn_pred":"tn"})
df_res_menos2.head()

780


,product_id,tn
0,20001,1504.68856
1,20002,1087.30855
2,20003,892.50129
3,20004,637.90002
4,20005,593.24443


In [24]:
df_res_menos2.to_csv(carpeta_base_exp + "naif_menos2_prediccion.csv",index=False)